# SymPy Exercises

In [2]:
import sympy as sym
from sympy import init_session
import math
import numpy as np
init_session()

IPython console for SymPy 1.2 (Python 3.7.2-64-bit) (ground types: gmpy)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at http://docs.sympy.org/1.2/



## Q1: Creating an expression

Create the expression:
$$f = x e^{-x} + x (1-x)$$
Then evaluate it for 
$$x = 0, 0.1, 0.2, 0.4, 0.8$$

In [6]:
e = symbols('e')
f = x*e**(-x)+x*(1-x)
print(f)
for val in [0,0.1,0.2,0.4,0.8]:
    a = f.subs([(x,val),(e,np.e)])
    print(a.evalf())


    

x*(-x + 1) + e**(-x)*x
0
0.180483741803596
0.323746150615596
0.508128018414256
0.519463171293777


## Q2: Factoring a polynomial, and finding its roots

Factor $$x^{4} - 6 x^{3} + x^{2} + 24 x + 16$$

Then find its zeros.

In [10]:
f = x**4 - 6*x**3 + x**2 + 24*x + 16
print(f)
f.factor()

expr = factor(x**4 - 6*(x**3) + (x**2) + 24*x + 16)
print(expr)
print(solve([expr], x))

x = sym.symbols("x")
f = x**4 - 6*x**3 + x**2 + 24*x + 16
print(solveset(f, x))
print(roots(f))

x**4 - 6*x**3 + x**2 + 24*x + 16
(x - 4)**2*(x + 1)**2
[(-1,), (4,)]
{-1, 4}
{4: 2, -1: 2}


## Q3: Integratation and differentiation

Integrate the function:
$$
f = \sin(x) e^{-x}
$$

Then differentiate the result to see if you get back the original function

In [12]:
f = sin(x)*exp(-x)
fint = integrate(f, x)
print(fint)
dfint = diff(fint, x)
print(dfint)
simplify(dfint-f)

-exp(-x)*sin(x)/2 - exp(-x)*cos(x)/2
exp(-x)*sin(x)


## Q4: Parsing an expression

Write a program that reads in a mathematical expression as a string (e.g., `"sin(2*pi*x)"`), converts it to a SymPy expression, and then evaluates it as needed.  

Have your program either make a plot of the entered function, or use the input function as the function to fit a dataset to using curvefit.

The following will be helpful:

`parse_expr()` will convert a string into a SymPy expression

In [14]:
from sympy.parsing.sympy_parser import parse_expr

In [20]:
s = "sin(2*pi*x)"
#print(type(s))
a = parse_expr(s)
print(a)
#print(type(a))


sin(2*pi*x)


`sympy.lambdify()` will convert a SymPy expression into a function that is callable by python.  You can make it a numpy-compatible function too (this means, e.g., that any `sin()` in your SymPy expression will be evaluate using `np.sin()`)

In [4]:
f = sym.lambdify(x, a, "numpy")

In [5]:
f(1.0)

In [6]:
help(lambdify)

Help on function lambdify in module sympy.utilities.lambdify:

lambdify(args, expr, modules=None, printer=None, use_imps=True, dummify=True)
    Returns a lambda function for fast calculation of numerical values.
    
    If not specified differently by the user, SymPy functions are replaced as
    far as possible by either python-math, numpy (if available) or mpmath
    functions - exactly in this order. To change this behavior, the "modules"
    argument can be used. It accepts:
    
     - the strings "math", "mpmath", "numpy", "numexpr", "sympy"
     - any modules (e.g. math)
     - dictionaries that map names of sympy functions to arbitrary functions
     - lists that contain a mix of the arguments above, with higher priority
       given to entries appearing first.
    
    The default behavior is to substitute all arguments in the provided
    expression with dummy symbols. This allows for applied functions (e.g.
    f(t)) to be supplied as arguments. Call the function with dumm

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def eval_expr(s, xval, plot=False, fit=False, ydata=[], funcvars=(), fitparams=[]):
    '''
    Reads in a string s and parses as a sympy expression to be evaluated at an array xval.
    If plot and fit are false, returns the array of function values.
    If plot is true and fit is false, plots the xval and f(xval) arrays as x and y.
    If plot if false and fit is true, it depends on inputting ydata to fit to. 
        It fits accordings to the parsed function and just returns the parameters and covariance matrix.
    If both are true, it plots the graph of the fit and returns the parameters and covariance matrix.
    
    funcvars is a tuple, and depends on the fit parameters having already been made symbols in sympy
        before the call, or else the string parsing won't be able to create the expression.
    fitparams is a list of initial guesses to whatever fit parameters you've included.
    '''
    a = parse_expr(s)
    if plot == True and fit == False:
        f = sym.lambdify(x, a, 'numpy')
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.plot(xval, f(xval))
        plt.show()
        return
    elif plot == True and fit == True:
        f = sym.lambdify(funcvars, a, 'numpy')
        popt, pcov = curve_fit(f, xval, ydata, p0=fitparams)
        perr = np.sqrt(np.diag(pcov))
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.plot(xval, f(xval, *popt), linestyle='--', label="Fit")
        ax.scatter(xval, ydata, label="Data", marker='x', color='k')
        ax.legend()
        for i in range(len(popt)):
            print('parameter {} = {} +- {}'.format(i, popt[i], perr[i]))
        plt.show()
        return (popt, pcov)
    elif plot == False and fit == True:
        f = sym.lambdify(funcvars, a, 'numpy')
        popt, pcov = curve_fit(f, xval, ydata, p0=fitparams)
        return (popt, pcov)
    else:
        f = sym.lambdify(x, a, 'numpy')
        return f(xval)

xvals = np.linspace(-10, 10, 100)
yvals = np.exp(-(xvals-0.5*np.random.randn(len(xvals)))**2)
A, sig, mu = symbols('A sig mu')
eval_expr("A*exp(-((x-mu)**2)/sig)", xvals, plot=True, fit=True, ydata=yvals, funcvars=(x, A, sig, mu),
         fitparams=[1.0, 0.5, 1.0])



In [21]:
from sympy.parsing.sympy_parser import parse_expr

def converter(str, lower, upper):
    """Converts a string str into a SymPy expression and plots the corresponding function 
    between integers lower and upper
    input: string str, integers lower and upper
    output: no output, a plot is created"""
    a = parse_expr(str)
    f = sym.lambdify(x, a, "numpy")
    t = np.linspace(lower,upper, num=100)
    plt.plot(t, f(t))
    plt.xlabel(r"$x$")
    plt.ylabel(str, fontsize="x-large")
    
converter("x**3", 0,5)


## Q5: Units

SymPy can deal with physical units.  See:

http://docs.sympy.org/latest/modules/physics/units/quantities.html

Let's try this out.  Newton's 2nd law is
$$F = ma$$

Create a mass of 1 kg and an acceleration of 10 m/s$^2$, and compute the force, $F$, and express the result in Newtons.

Note: the `convert_to` function was added in SymPy 1.1, so if you are using an earlier version, you will need to divide by the target unit to do the conversion.

In [23]:
from sympy.physics.units import kilogram, newton, meter, second
from sympy.physics.units import convert_to
F, m, a = symbols('F m a')
m = 1*kilogram
a = 10*meter/second**2
F = m*a
convert_to(F,newton)